In [6]:
import requests
from requests.auth import HTTPBasicAuth

In [11]:
import random

In [29]:
import json

In [3]:
import os

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [23]:
host = 'http://localhost:5000/'

In [17]:
auth=HTTPBasicAuth(os.environ.get('BASIC_AUTH_USERNAME'), os.environ.get('BASIC_AUTH_PASSWORD') )

In [18]:
requests.get(host, auth=auth )

<Response [200]>

In [67]:
data = {
    'execId': 'exec' + str( random.randint(1000, 10000000) ),
    'orderId': str( random.randint(1000, 10000000) ),
    'time': '20180604 8:40',
    'acctNumber': 'U' + str( random.randint(1000, 10000000) ),
    'exchange': 'CBOE',
    'side': 'BOT',
    'shares': 13,
    'cumQty': -13,
    'price': 7.23,
    'avgPrice': -7.23,
    'permId': str( random.randint(1000, 10000000) ),
    
    'localSymbol': 'GOOGL 6 C1200.25 Equity',
    'secType': 'OPT',
    'symbol': 'GOOGL',
    'currency': 'USD',
    'primaryExchange': '',
    'lastTradeDateOrcontractMonth': '20180615',
    'multiplier': 100,
    'strike': 1200.25,
    'right': 'C'
    
}
data

{'execId': 'exec4724913',
 'orderId': '8765152',
 'time': '20180604 8:40',
 'acctNumber': 'U5066353',
 'exchange': 'CBOE',
 'side': 'BOT',
 'shares': 13,
 'cumQty': -13,
 'price': 7.23,
 'avgPrice': -7.23,
 'permId': '2720843',
 'localSymbol': 'GOOGL 6 C1200.25 Equity',
 'secType': 'OPT',
 'symbol': 'GOOGL',
 'currency': 'USD',
 'primaryExchange': '',
 'lastTradeDateOrcontractMonth': '20180615',
 'multiplier': 100,
 'strike': 1200.25,
 'right': 'C'}

In [81]:
data2 = {
    'execId': 'exec' + str( random.randint(1000, 10000000) ),
    'orderId': str( random.randint(1000, 10000000) ),
    'time': '20180604 8:40',
    'acctNumber': 'U' + str( random.randint(1000, 10000000) ),
    'exchange': 'CBOE',
    'side': 'BOT',
    'shares': 13,
    'cumQty': -13,
    'price': 7.23,
    'avgPrice': -7.23,
    'permId': str( random.randint(1000, 10000000) ),
    
    'localSymbol': 'GOOGL 6 C' + str( random.randint(900, 1400) ) + '.25 Equity',
    'secType': 'OPT',
    'symbol': 'GOOGL',
    'currency': 'USD',
    'primaryExchange': '',
    'lastTradeDateOrcontractMonth': '20180615',
    'multiplier': 100,
    'strike': 1200.25,
    'right': 'C'
    
}

res = requests.post(host + 'executions', auth=auth,json=data2 )
#res.content
json.loads(res.content)

{'contract': {'localSymbol': 'GOOGL 6 C1397.25 Equity'},
 'data': {'acctNumber': 'U9767530',
  'avgPrice': -7.23,
  'cumQty': -13,
  'currency': 'USD',
  'exchange': 'CBOE',
  'execId': 'exec6403726',
  'lastTradeDateOrcontractMonth': '20180615',
  'localSymbol': 'GOOGL 6 C1397.25 Equity',
  'multiplier': 100,
  'orderId': '2539300',
  'permId': '7840157',
  'price': 7.23,
  'primaryExchange': '',
  'right': 'C',
  'secType': 'OPT',
  'shares': 13,
  'side': 'BOT',
  'strike': 1200.25,
  'symbol': 'GOOGL',
  'time': '20180604 8:40'},
 'execution': {'execId': 'exec6403726'},
 'order': {'orderId': 2539300},
 'status': 'ok'}